<a href="https://colab.research.google.com/github/yubinvip/Hung-yi-Lee-AI2024/blob/main/GenAI_HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 7 - Understand what Generative AI is thinking

In [ ]:
!pip install inseq==0.5.0
!pip install transformers
!pip install bitsandbytes
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.7/136.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 569.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 20.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.

## Machine Translation Task

In [ ]:
import inseq

# List of attribution methods to be used
attribution_methods = ['saliency', 'attention']

for method in attribution_methods:
    print(f"======= Attribution Method: {method} =======")
    # Load the Chinese-to-English translation model and set up the attribution method
    model = inseq.load_model("Helsinki-NLP/opus-mt-zh-en", method)

    # Apply attribution to the input text using the specified method
    attribution_result = model.attribute(
        input_texts="我喜歡機器學習和人工智慧。",
        step_scores=["probability"],
    )

    # Remove '▁' from the tokenizer in the prefix to avoid confusion (You can ignore this part of code)
    for attr in attribution_result.sequence_attributions:
        for item in attr.source:
            item.token = item.token.replace('▁', '')
        for item in attr.target:
            item.token = item.token.replace('▁', '')

    # Display the attribution results
    attribution_result.show()

## Sentence Completion Task

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2-xl", load_in_8bit=True, device_map="auto")

In [ ]:
for method in attribution_methods:
    # Load the model with the specified attribution method
    inseq_model = inseq.load_model(model, method)

    # Apply attribution to the input text using the specified method
    attribution_result = inseq_model.attribute(
        input_texts="The first president of America is",
        step_scores=["probability"],
    )

    # Remove 'Ġ' from GPT2's BPE tokenizer in the prefix to avoid confusion (You can ignore this part of code)
    for attr in attribution_result.sequence_attributions:
        for item in attr.source:
            item.token = item.token.replace('Ġ', '')
        for item in attr.target:
            item.token = item.token.replace('Ġ', '')

    # Display the attribution results
    attribution_result.show()